In [ ]:
!pip uninstall numpy
!pip install numpy==1.23.1



In [12]:
import random
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Activation
from tensorflow.keras.optimizers import RMSprop

#NOW download the shakespeare txt file
filepath = tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

text = open(filepath,'rb').read().decode(encoding='utf-8').lower()

text = text[300000:800000]

characters = sorted(set(text))

char_to_index = dict((c,i) for i,c in enumerate(characters))
index_to_char = dict((i,c) for i,c in enumerate(characters)) 

SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_characters = []

for i in range(0,len(text) - SEQ_LENGTH, STEP_SIZE):
	sentences.append(text[i: i+SEQ_LENGTH])
	next_characters.append(text[i+SEQ_LENGTH])
	
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=np.bool_)
y = np.zeros((len(sentences),len(characters)), dtype=np.bool_)

for i, sentence in enumerate(sentences):
	for t, character in enumerate(sentence):
		x[i,t,char_to_index[character]]=1
	y[i,char_to_index[next_characters[i]]]=1


#NOW we will take the 2 arrays (x and y) and feed them to a RNN model

model = Sequential()
model.add(LSTM(128,input_shape=(SEQ_LENGTH,len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='Adam')

model.fit(x,y,batch_size=256,epochs=4)




Epoch 1/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 75s 112ms/step - loss: 2.9133
Epoch 2/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 82s 113ms/step - loss: 2.2613
Epoch 3/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 73s 113ms/step - loss: 2.1232
Epoch 4/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 82s 113ms/step - loss: 2.0259


In [19]:
#Saving the model
#model.save('textgenerator.model')
#-------------
#NOW load the saved model
#model = tf.keras.model.load_model('textgenerator.model')

#Now creating a function that predicts

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1,preds,1)
    return np.argmax(probas)



def generate_text(length,temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x = np.zeros((1,SEQ_LENGTH, len(characters)))
        for t, character in enumerate(sentence):
            x[0,t,char_to_index[character]] = 1
        
        predictions = model.predict(x,verbose=0)[0]
        next_index = sample(predictions,temperature)
        next_character = index_to_char[next_index]
        
        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

print('--------0.2---------')
print(generate_text(300, 0.2))
print('--------0.4---------')
print(generate_text(300, 0.4))
print('--------0.6---------')
print(generate_text(300, 0.6))
print('--------0.8---------')
print(generate_text(300, 0.8))
print('---------1----------')
print(generate_text(300, 1.0))

--------0.2---------
my hand,
that i yet know not?

friar laucer:
the pare the sall the ware the seath the manter the seath the sore the sord the here the reather the for the seath the great the will the sore the for the wert the hare the merst the mare the the mant the beat of the manter the reare the beat the hard and the manter the manter and the will the 
--------0.4---------
t, i'll give thee remedy.

juliet:
o, bith to me the prece on were of the will the erent that the is steed the prace,
thou the wart of here the for the count
the make that the will the roust and my but you har the ere the profice on the beathes and me that for the sters bo more bronger with the hist the will for what dis tay and the pray 
--------0.6---------
lack'd sight only, nought for approbation grionge with arist that not reath,
thesh the till be that the dabed patt of arimen sow with apy the hes in that so gand maithy lewand thou bropoon liggrong of the with the shall me sull her, broffor the foren be f